# Data preparation


- Join CSV files
- brief overview about the data so it can be cleaned and preprocessed
- Cleaning and preprocessing
    - rename columns 
    - convert datatypes
    - Duplicates
    - Data Aggreagation
    - impute missing values


In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split
import pandas as pd
import os


## Join Data
In this chapter we jon the CSV files that have data about car accidents in germany in a time span from 2016 to 2022. The source of the data is https://unfallatlas.statistikportal.de/

In [2]:

#merge Traffic Accident Data from 2016 to 2022
folder_path = "C:\Projekte\TDS\TDS2324-TrafficAccidents\Data\TrafficAccidentData"
file_list = os.listdir(folder_path)
merged_data = pd.DataFrame()

# Iterate over each file in the folder and add to merged_data
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    data = pd.read_csv(file_path,sep=';')
    merged_data = pd.concat([merged_data, data], ignore_index=True)

print(merged_data.columns)

C:\Users\miche\AppData\Local\Temp\ipykernel_18672\3181226627.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path,sep=';')
C:\Users\miche\AppData\Local\Temp\ipykernel_18672\3181226627.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path,sep=';')


Index(['FID', 'OBJECTID', 'ULAND', 'UREGBEZ', 'UKREIS', 'UGEMEINDE', 'UJAHR',
       'UMONAT', 'USTUNDE', 'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1',
       'ULICHTVERH', 'STRZUSTAND', 'IstRad', 'IstPKW', 'IstFuss', 'IstKrad',
       'IstGkfz', 'IstSonstige', 'LINREFX', 'LINREFY', 'XGCSWGS84',
       'YGCSWGS84', 'UIDENTSTLA', 'OBJECTID_1', 'UIDENTSTLAE'],
      dtype='object')


## Introducing the data set

Before we start cleaning and pre-processing the data, we create a brief overview 

In [3]:
merged_data.head(10)

,FID,OBJECTID,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,...,IstKrad,IstGkfz,IstSonstige,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84,UIDENTSTLA,OBJECTID_1,UIDENTSTLAE
0,0.0,1.0,1,0,53,120,2016,1,9,5,...,0,0.0,0,"606982,393999999970000","5954659,924999999800000","10,621659329000000","53,729614888000000",NaN,NaN,NaN
1,1.0,2.0,1,0,57,10,2016,1,17,3,...,0,0.0,0,"574882,533000000050000","6011440,850999999800000","10,149175783000000","54,245452583999999",NaN,NaN,NaN
2,2.0,3.0,1,0,62,8,2016,1,0,5,...,0,0.0,0,"599934,687500000000000","5964608,596199999600000","10,518094344000000","53,820402504999997",NaN,NaN,NaN
3,3.0,4.0,1,0,3,0,2016,1,15,5,...,0,0.0,1,"610709,348699999970000","5968284,242300000000000","10,683020702000000","53,851243101000001",NaN,NaN,NaN
4,4.0,5.0,1,0,55,28,2016,1,14,1,...,0,0.0,0,"605690,790400000060000","6009152,214700000400000","10,620986018000000","54,219458582000001",NaN,NaN,NaN
5,5.0,6.0,1,0,61,44,2016,1,17,6,...,0,0.0,0,"542547,328499999950000","5963921,747000000400000","9,646338030000001","53,822093844999998",NaN,NaN,NaN
6,6.0,7.0,1,0,60,54,2016,1,17,3,...,0,0.0,0,"558412,407499999970000","5969889,909599999900000","9,888452573000000","53,874189672000000",NaN,NaN,NaN
7,7.0,8.0,1,0,55,29,2016,1,11,1,...,0,0.0,0,"623911,704400000050000","6008211,261900000300000","10,899895740000000","54,206924608999998",NaN,NaN,NaN
8,8.0,9.0,1,0,60,85,2016,1,16,2,...,0,0.0,0,"579664,308100000020000","5965471,277499999900000","10,210478533000000","53,831655902000001",NaN,NaN,NaN
9,9.0,10.0,1,0,54,126,2016,1,10,4,...,0,0.0,1,"494320,527400000020000","6066809,662200000100000","8,911765602999999","54,748506792999997",NaN,NaN,NaN


In [4]:
merged_data.shape

(1554834, 28)

In [5]:
merged_data.describe()

,FID,OBJECTID,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,...,UTYP1,ULICHTVERH,STRZUSTAND,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,OBJECTID_1
count,151673.000000,1.342966e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,...,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.554834e+06,1.359605e+06,1.554834e+06,211868.000000
mean,75836.000000,1.154006e+05,7.431094e+00,2.788656e+00,4.191415e+01,6.952885e+01,2.019268e+03,6.766878e+00,1.331008e+01,4.102703e+00,...,3.819214e+00,4.294851e-01,2.764469e-01,2.878140e-01,7.866357e-01,8.647933e-02,1.412794e-01,4.938934e-02,1.111977e-01,105934.500000
std,43784.368027,6.982492e+04,3.588564e+00,2.349308e+00,2.588040e+01,1.116782e+02,1.918956e+00,3.214572e+00,4.776594e+00,1.868558e+00,...,2.140342e+00,7.880731e-01,4.945796e-01,4.527441e-01,4.096829e-01,2.810706e-01,3.483096e-01,2.166797e-01,3.143769e-01,61161.167754
min,0.000000,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.016000e+03,1.000000e+00,0.000000e+00,1.000000e+00,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
25%,37918.000000,5.595725e+04,5.000000e+00,1.000000e+00,1.500000e+01,0.000000e+00,2.018000e+03,4.000000e+00,1.000000e+01,3.000000e+00,...,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,52967.750000
50%,75836.000000,1.119140e+05,8.000000e+00,3.000000e+00,3.900000e+01,2.000000e+01,2.019000e+03,7.000000e+00,1.400000e+01,4.000000e+00,...,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,105934.500000
75%,113754.000000,1.711110e+05,9.000000e+00,4.000000e+00,6.300000e+01,1.130000e+02,2.021000e+03,9.000000e+00,1.700000e+01,6.000000e+00,...,6.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,158901.250000
max,151672.000000,2.683700e+05,1.600000e+01,9.000000e+00,9.100000e+01,7.180000e+02,2.022000e+03,1.200000e+01,2.300000e+01,7.000000e+00,...,7.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,211868.000000


In [6]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554834 entries, 0 to 1554833
Data columns (total 28 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   FID          151673 non-null   float64
 1   OBJECTID     1342966 non-null  float64
 2   ULAND        1554834 non-null  int64  
 3   UREGBEZ      1554834 non-null  int64  
 4   UKREIS       1554834 non-null  int64  
 5   UGEMEINDE    1554834 non-null  int64  
 6   UJAHR        1554834 non-null  int64  
 7   UMONAT       1554834 non-null  int64  
 8   USTUNDE      1554834 non-null  int64  
 9   UWOCHENTAG   1554834 non-null  int64  
 10  UKATEGORIE   1554834 non-null  int64  
 11  UART         1554834 non-null  int64  
 12  UTYP1        1554834 non-null  int64  
 13  ULICHTVERH   1554834 non-null  int64  
 14  STRZUSTAND   1554834 non-null  int64  
 15  IstRad       1554834 non-null  int64  
 16  IstPKW       1554834 non-null  int64  
 17  IstFuss      1554834 non-null  int64  
 18  Is

# Cleaning and preprocessing

### Drop Data
features being droped that are unnecessay for this project since they are partly foreign keys or ids that are not needed or named in the different datasets thta where joined differently

In [7]:
merged_data.drop(['FID', 'OBJECTID', 'OBJECTID_1', 'UIDENTSTLA', 'UIDENTSTLAE'], axis=1, inplace=True)
print(merged_data.columns)
#save the merged data to a csv file in folder TrafficAccidentData



Index(['ULAND', 'UREGBEZ', 'UKREIS', 'UGEMEINDE', 'UJAHR', 'UMONAT', 'USTUNDE',
       'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1', 'ULICHTVERH', 'STRZUSTAND',
       'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstige',
       'LINREFX', 'LINREFY', 'XGCSWGS84', 'YGCSWGS84'],
      dtype='object')


### Convert datatypes
Since the coordinates are saved as Strings we  have to convert them in a datatype we can later work with. 

In [8]:
merged_data['XGCSWGS84'] = merged_data['XGCSWGS84'].astype(str).str.replace(',', '.').astype(float)
merged_data['YGCSWGS84'] = merged_data['YGCSWGS84'].astype(str).str.replace(',', '.').astype(float)
merged_data['LINREFX'] = merged_data['LINREFX'].astype(str).str.replace(',', '.').astype(float)
merged_data['LINREFY'] = merged_data['LINREFY'].astype(str).str.replace(',', '.').astype(float)

merged_data.head(10)


,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,...,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,1,0,53,120,2016,1,9,5,2,8,...,0,1,0,0,0.0,0,606982.3940,5.954660e+06,10.621659,53.729615
1,1,0,57,10,2016,1,17,3,3,1,...,0,1,0,0,0.0,0,574882.5330,6.011441e+06,10.149176,54.245453
2,1,0,62,8,2016,1,0,5,3,9,...,0,1,0,0,0.0,0,599934.6875,5.964609e+06,10.518094,53.820403
3,1,0,3,0,2016,1,15,5,3,5,...,1,0,0,0,0.0,1,610709.3487,5.968284e+06,10.683021,53.851243
4,1,0,55,28,2016,1,14,1,3,8,...,0,1,0,0,0.0,0,605690.7904,6.009152e+06,10.620986,54.219459
5,1,0,61,44,2016,1,17,6,3,9,...,0,1,0,0,0.0,0,542547.3285,5.963922e+06,9.646338,53.822094
6,1,0,60,54,2016,1,17,3,3,6,...,1,0,1,0,0.0,0,558412.4075,5.969890e+06,9.888453,53.874190
7,1,0,55,29,2016,1,11,1,2,2,...,0,1,0,0,0.0,0,623911.7044,6.008211e+06,10.899896,54.206925
8,1,0,60,85,2016,1,16,2,2,4,...,0,1,0,0,0.0,0,579664.3081,5.965471e+06,10.210479,53.831656
9,1,0,54,126,2016,1,10,4,2,5,...,0,1,0,0,0.0,1,494320.5274,6.066810e+06,8.911766,54.748507


In [9]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554834 entries, 0 to 1554833
Data columns (total 23 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   ULAND        1554834 non-null  int64  
 1   UREGBEZ      1554834 non-null  int64  
 2   UKREIS       1554834 non-null  int64  
 3   UGEMEINDE    1554834 non-null  int64  
 4   UJAHR        1554834 non-null  int64  
 5   UMONAT       1554834 non-null  int64  
 6   USTUNDE      1554834 non-null  int64  
 7   UWOCHENTAG   1554834 non-null  int64  
 8   UKATEGORIE   1554834 non-null  int64  
 9   UART         1554834 non-null  int64  
 10  UTYP1        1554834 non-null  int64  
 11  ULICHTVERH   1554834 non-null  int64  
 12  STRZUSTAND   1554834 non-null  int64  
 13  IstRad       1554834 non-null  int64  
 14  IstPKW       1554834 non-null  int64  
 15  IstFuss      1554834 non-null  int64  
 16  IstKrad      1554834 non-null  int64  
 17  IstGkfz      1359605 non-null  float64
 18  Is

### Checking for Duplicates
Checking for duplicates by making a copy and doubling our dataset. Afterwards the doubles are being dropped. Are there no douplictates in the copied dataset, the original doesnt have duplicates either.

In [10]:
temp_df = merged_data.copy()
temp_df.drop_duplicates(inplace=True)
num_rows_merged_data = merged_data.shape[0]
num_rows_temp_df = temp_df.shape[0]

print("number of rows in merged_data: ", num_rows_merged_data)
print("number of rows in temp_df: ", num_rows_temp_df)    
print("difference in rows: ", num_rows_temp_df - num_rows_merged_data)

number of rows in merged_data:  1554834
number of rows in temp_df:  1554694
difference in rows:  -140


Since there is a difference of 140 we it is probible to have duplicates in our original dataset.

In [11]:
# show duplicates 
duplicates = merged_data[merged_data.duplicated(keep=False)]
duplicates.head(14)

,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,...,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
52335,7,3,39,43,2016,3,13,4,3,1,...,0,1,0,0,0.0,0,453653.9008,5.523542e+06,8.355149,49.862495
52339,7,3,39,43,2016,3,13,4,3,1,...,0,1,0,0,0.0,0,453653.9008,5.523542e+06,8.355149,49.862495
54140,7,1,37,69,2016,6,5,5,3,2,...,0,1,0,0,0.0,0,376200.4225,5.583253e+06,7.258458,50.388285
54478,7,1,37,69,2016,6,5,5,3,2,...,0,1,0,0,0.0,0,376200.4225,5.583253e+06,7.258458,50.388285
58019,7,1,43,59,2016,9,14,5,3,2,...,0,1,0,0,0.0,0,409948.5422,5.595250e+06,7.730177,50.502286
58247,7,1,43,59,2016,9,14,5,3,2,...,0,1,0,0,0.0,0,409948.5422,5.595250e+06,7.730177,50.502286
58314,7,3,12,0,2016,10,21,7,3,9,...,0,1,0,1,0.0,0,406861.1510,5.478158e+06,7.715006,49.448939
58484,7,3,12,0,2016,10,21,7,3,9,...,0,1,0,1,0.0,0,406861.1510,5.478158e+06,7.715006,49.448939
60015,7,3,38,6,2016,12,19,1,3,2,...,0,1,0,0,0.0,0,452163.5267,5.473644e+06,8.340503,49.413595
60256,7,3,38,6,2016,12,19,1,3,2,...,0,1,0,0,0.0,0,452163.5267,5.473644e+06,8.340503,49.413595


The table shows that there are indeed duplicates in the dataset. Since the coordinates are exremly detailed, it is very unlilkely that the accdients happend the same time with the same particitpantson the exact same spot. After doublechecking the accidents on the map we decided to drop them.

In [12]:
#drop duplicates
merged_data.drop_duplicates(inplace=True)
num_rows_merged_data = merged_data.shape[0]

print("number of rows in merged_data: ", num_rows_merged_data)
print("number of rows in temp_df: ", num_rows_temp_df)
print("difference in rows: ", num_rows_temp_df - num_rows_merged_data)


number of rows in merged_data:  1554694
number of rows in temp_df:  1554694
difference in rows:  0


Since they have now the same amount of row, there are no further duplicates in the dataset.


### Renaming columns

In [13]:
merged_data.columns

Index(['ULAND', 'UREGBEZ', 'UKREIS', 'UGEMEINDE', 'UJAHR', 'UMONAT', 'USTUNDE',
       'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1', 'ULICHTVERH', 'STRZUSTAND',
       'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstige',
       'LINREFX', 'LINREFY', 'XGCSWGS84', 'YGCSWGS84'],
      dtype='object')

In [14]:
# Cols in lower case   
merged_data.columns =  [col.lower() for col in merged_data.columns]
merged_data.columns



Index(['uland', 'uregbez', 'ukreis', 'ugemeinde', 'ujahr', 'umonat', 'ustunde',
       'uwochentag', 'ukategorie', 'uart', 'utyp1', 'ulichtverh', 'strzustand',
       'istrad', 'istpkw', 'istfuss', 'istkrad', 'istgkfz', 'istsonstige',
       'linrefx', 'linrefy', 'xgcswgs84', 'ygcswgs84'],
      dtype='object')

Since the U in the colnames stands for 'Unfall' and its obvious that we are having a dataset with data of accidents the U will be deleted to make things clearer. Furthermore the 1 in typ will be deleted and a _ will be added for the boolean categories.

In [15]:
merged_data.columns = merged_data.columns.str.lstrip('u')
merged_data.rename(columns=lambda x: x.replace('ist', 'ist_') if x.startswith('ist') else x, inplace=True)
merged_data.rename(columns={
    'typ1': 'typ',
    'lichtverh': 'licht' 
    }, inplace=True)

merged_data.columns


Index(['land', 'regbez', 'kreis', 'gemeinde', 'jahr', 'monat', 'stunde',
       'wochentag', 'kategorie', 'art', 'typ', 'licht', 'strzustand',
       'ist_rad', 'ist_pkw', 'ist_fuss', 'ist_krad', 'ist_gkfz',
       'ist_sonstige', 'linrefx', 'linrefy', 'xgcswgs84', 'ygcswgs84'],
      dtype='object')

### Data Aggregation
Since it is easier for the analysis later to search for cities with the Amtliche Gemeinde Schlüssel (AGS) we are aggregating this feature from the features land, regbez, kreis, gemeinde.

In [16]:
# Funktion zur Erstellung des AGS
def create_ags(row):
    land = f"{int(row['land']):02d}"  
    regbez = f"{int(row['regbez']):01d}"
    kreis = f"{int(row['kreis']):02d}"  
    gemeinde = f"{int(row['gemeinde']):03d}"  
    return land + regbez + kreis + gemeinde

# Anwenden der Funktion auf jede Zeile und Hinzufügen einer neuen Spalte "AGS"
merged_data['ags'] = merged_data.apply(create_ags, axis=1).astype(str)

# Ausgabe der ersten 5 Zeilen mit dem neuen AGS zur Überprüfung
print(merged_data[['land', 'regbez', 'kreis', 'gemeinde', 'ags']].sample(20))


         land  regbez  kreis  gemeinde       ags
1049117     1       0      1         0  01001000
89462       9       6     79       180  09679180
1004047     4       0     11         0  04011000
282398      3       4     61         3  03461003
1547626    14       7     13         0  14713000
156047      1       0     53        46  01053046
551803     11       0      9         9  11009009
282372      3       2     41         1  03241001
562861      1       0     60        68  01060068
1320437     3       3     52        32  03352032
1437670    15       0     88       330  15088330
285620      3       3     60        25  03360025
655032      5       7     74        32  05774032
500756      8       2     35        66  08235066
490918      6       4     39        15  06439015
1376403     7       1     35        18  07135018
598452      8       2     15        89  08215089
967234     11       0      2         2  11002002
205695      8       2     31         0  08231000
376717      6       

### Missing Values

In [17]:
# check for missing values
merged_data.isnull().sum()



land                 0
regbez               0
kreis                0
gemeinde             0
jahr                 0
monat                0
stunde               0
wochentag            0
kategorie            0
art                  0
typ                  0
licht                0
strzustand           0
ist_rad              0
ist_pkw              0
ist_fuss             0
ist_krad             0
ist_gkfz        195214
ist_sonstige         0
linrefx              0
linrefy              0
xgcswgs84            0
ygcswgs84            0
ags                  0
dtype: int64

As we can see the data set has almost no null values except for the binary variable 'ist_gkfz'. This will be investigated further and the data imputed where appropriate.

In [18]:
missing_data = merged_data[merged_data['ist_gkfz'].isnull()]
missing_data.sample(10)

,land,regbez,kreis,gemeinde,jahr,monat,stunde,wochentag,kategorie,art,...,ist_pkw,ist_fuss,ist_krad,ist_gkfz,ist_sonstige,linrefx,linrefy,xgcswgs84,ygcswgs84,ags
310738,14,7,13,0,2017,1,17,5,3,5,...,1,0,0,NaN,0,739759.4850,5.688233e+06,12.439147,51.294858,14713000
314910,14,6,12,0,2017,10,14,2,3,2,...,1,0,0,NaN,0,836983.1199,5.666446e+06,13.808810,51.050461,14612000
178525,6,6,33,17,2017,1,14,5,3,2,...,1,0,0,NaN,0,536960.1515,5.681114e+06,9.529925,51.280162,06633017
218361,15,0,89,30,2017,1,6,4,3,6,...,1,1,0,NaN,0,688984.8449,5.741399e+06,11.740385,51.791483,15089030
255653,7,1,33,111,2017,6,17,6,3,9,...,1,0,0,NaN,0,396881.5600,5.517376e+06,7.567058,49.799973,07133111
155053,1,0,2,0,2017,6,8,4,3,5,...,1,0,0,NaN,0,573162.1870,6.021789e+06,10.125313,54.338686,01002000
191783,8,2,12,0,2017,1,12,1,3,8,...,1,0,0,NaN,1,452686.1432,5.429151e+06,8.352952,49.013434,08212000
247041,9,6,73,141,2017,10,10,1,1,8,...,1,0,0,NaN,0,601636.3841,5.570853e+06,10.426534,50.281052,09673141
194745,8,3,35,43,2017,3,7,5,3,2,...,0,0,0,NaN,0,513842.6721,5.279558e+06,9.184394,47.669268,08335043
215531,14,5,22,240,2017,7,10,3,2,5,...,1,0,0,NaN,0,811670.2660,5.656501e+06,13.440262,50.975589,14522240


In [19]:
missing_data_2017 = merged_data[merged_data['ist_gkfz'].isnull() & (merged_data['jahr'] == 2017)]
all_null_from_2017 = len(missing_data_2017) == len(missing_data)
not_null_from_2017 = merged_data[merged_data['ist_gkfz'].notnull() & merged_data['jahr'] == 2017].any().any()
print("All the null values for feature is_gkfz are from year 2017: ", all_null_from_2017)
print("There are notnull values for ist_gkfz in year 2017: ", not_null_from_2017)



All the null values for feature is_gkfz are from year 2017:  True
There are notnull values for ist_gkfz in year 2017:  False


Since all null values are only for the feature is_gkfz from 2017 and there are no notnull values for this feature in this year, it can be concluded that no data was collected for is_gkfz in this year.

In the following we will impute the missing values. We will check for patterns and correlation with other features. So we can train a logistic regression model, that imputes our missing values for the year 2017 

In [20]:
# Calculate the correlation matrix
correlation_matrix = merged_data.corr()

# Select the features with high correlation with is_gkfz
relevant_features = correlation_matrix['ist_gkfz'].abs().sort_values(ascending=False).index[1:]

# map correlation to each feature
correlation_to_ist_gkfz = {feature: correlation_matrix.loc['ist_gkfz', feature] for feature in relevant_features}


# Print the relevant features
print(correlation_to_ist_gkfz)


{'ist_rad': -0.10733862747940964, 'typ': 0.09456375198947359, 'stunde': -0.07848527764753568, 'ist_krad': -0.057456393171918876, 'ist_pkw': -0.057187447091856254, 'kategorie': -0.0519069042703419, 'ist_fuss': -0.046028361121884936, 'art': -0.0393140192823976, 'kreis': 0.02431316433109911, 'licht': -0.019836170951205146, 'gemeinde': 0.018754307961221503, 'wochentag': -0.013051362686227398, 'monat': -0.00970261731071049, 'jahr': -0.00842786096139455, 'linrefy': -0.0062591436233858395, 'ags': 0.0062353802525829845, 'ygcswgs84': -0.006115356146857279, 'land': 0.005696697403717505, 'regbez': 0.005381606881152162, 'ist_sonstige': -0.00532538672720455, 'linrefx': 0.003561234236690129, 'strzustand': 0.00337820718997781, 'xgcswgs84': 0.0032319654037303116}


In [21]:
imputation_data = merged_data.copy()

# delete data with missing values
imputation_data.dropna(inplace=True)

#defining the features and target
features = imputation_data.drop(['ist_gkfz', 'jahr', 'monat', 'land', 'regbez','xgcswgs84', 'ygcswgs84', 'linrefx', 'linrefy'],  axis=1)
target = imputation_data['ist_gkfz']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create a scaler object
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train = scaler.fit_transform(X_train)

# Transform the test data
X_test = scaler.transform(X_test)

# Create a logistic regression model
log_reg = LogisticRegression(max_iter=1000)

# Fit the model
log_reg.fit(X_train, y_train)

# Calculate accuracy score on test data
accuracy_score = accuracy_score(log_reg.predict(X_test), y_test)
print(accuracy_score)
precision_score = precision_score(log_reg.predict(X_test), y_test)
print(precision_score)




0.9556264159825816
0.11289368623391986


In [22]:
# Select the rows with missing values for ist_gkfz
missing_data = merged_data[merged_data['ist_gkfz'].isnull()]

# Extract the features for imputation
imputation_features = missing_data.drop(['ist_gkfz', 'jahr', 'monat', 'land', 'regbez', 'xgcswgs84', 'ygcswgs84', 'linrefx', 'linrefy'], axis=1)

# Scale the features using the previously created scaler
imputation_features_scaled = scaler.transform(imputation_features)

# Impute the missing values using the logistic regression model
imputed_values = log_reg.predict(imputation_features_scaled)

# Assign the imputed values to the missing rows in the merged_data dataframe
merged_data.loc[merged_data['ist_gkfz'].isnull(), 'ist_gkfz'] = imputed_values

# Check if there are any remaining missing values
merged_data.isnull().sum()


land            0
regbez          0
kreis           0
gemeinde        0
jahr            0
monat           0
stunde          0
wochentag       0
kategorie       0
art             0
typ             0
licht           0
strzustand      0
ist_rad         0
ist_pkw         0
ist_fuss        0
ist_krad        0
ist_gkfz        0
ist_sonstige    0
linrefx         0
linrefy         0
xgcswgs84       0
ygcswgs84       0
ags             0
dtype: int64

saving the data as csv  

In [23]:
merged_data.to_csv("C:\\Projekte\\TDS\\TDS2324-TrafficAccidents\\Data\\all_16_22.csv")

In [24]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1554694 entries, 0 to 1554833
Data columns (total 24 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   land          1554694 non-null  int64  
 1   regbez        1554694 non-null  int64  
 2   kreis         1554694 non-null  int64  
 3   gemeinde      1554694 non-null  int64  
 4   jahr          1554694 non-null  int64  
 5   monat         1554694 non-null  int64  
 6   stunde        1554694 non-null  int64  
 7   wochentag     1554694 non-null  int64  
 8   kategorie     1554694 non-null  int64  
 9   art           1554694 non-null  int64  
 10  typ           1554694 non-null  int64  
 11  licht         1554694 non-null  int64  
 12  strzustand    1554694 non-null  int64  
 13  ist_rad       1554694 non-null  int64  
 14  ist_pkw       1554694 non-null  int64  
 15  ist_fuss      1554694 non-null  int64  
 16  ist_krad      1554694 non-null  int64  
 17  ist_gkfz      1554694 non-null  